In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from datetime import datetime
from datetime import timedelta, date
import matplotlib.dates as mdates
import math
import warnings

from sklearn import neighbors
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings('ignore')

#from xgboost import XGBClassifier
#from sklearn.metrics import accuracy_score

In [2]:
properties = pd.read_csv('trainTree.csv')
testing = pd.read_csv('testTree.csv')

In [3]:
#properties = properties.loc[:, ['id', 'place_name_encoded', 'autopista', 'baja', 'barrio cerrado', 'cancha', 'centro', 'cocina', 'contrafrente', 'cuadras', 'excelente', 'garage', 'gimnasio', 'grande', 'jardin', 'lavadero', 'limpieza', 'living', 'luminoso', 'madero', 'monoambiente', 'nuevo', 'parque', 'patio', 'pileta', 'playroom', 'seguridad', 'shopping', 'surface_covered_in_m2', 'terraza', 'tren', 'price_aprox_usd']]
#testing = testing.loc[:, ['id', 'place_name_encoded', 'autopista', 'baja', 'barrio cerrado', 'cancha', 'centro', 'cocina', 'contrafrente', 'cuadras', 'excelente', 'garage', 'gimnasio', 'grande', 'jardin', 'lavadero', 'limpieza', 'living', 'luminoso', 'madero', 'monoambiente', 'nuevo', 'parque', 'patio', 'pileta', 'playroom', 'seguridad', 'shopping', 'surface_covered_in_m2', 'terraza', 'tren']]

train = properties
train = train.drop('id', 1)
train = train.drop('description', 1)
train = train.drop('Unnamed: 0', 1)
train = train.drop('created_on', 1)
train = train.drop('lat', 1)
train = train.drop('lon', 1)

In [4]:
t = testing
t = t.drop('description',1)
t = t.drop('Unnamed: 0',1)
t = t.drop('created_on',1)
t = t.drop('lat',1)
t = t.drop('lon',1)
#t = t.drop('place_name_encoded',1)
#t = t.drop('id',1)

In [5]:
#obtengo trainvacios y testvacios con los entries de los respectivos datasets que cumplen:
#en el train set, menos de 10 entradas por place_name_encoded
#en el test set, no tienen entradas en el train set con el mismo place_name_encoded


l = len(properties['place_name_encoded'].value_counts())

#CAMBIO: hardcodeo el valor original (place name encoded va hasta 678)


#uso este porque sé que es bajo
trainvacios = train[train['place_name_encoded']==1]
testvacios = t[t['place_name_encoded']==1]

for x in range(0,l+2):
    if ((len(t[t['place_name_encoded']==x])==0) or (len(train[train['place_name_encoded']==x])<50)) and (x!=1):
    #excluyo el hardcodeado
    
        train0 = train[train['place_name_encoded']==x]
        trainvacios = pd.concat([trainvacios, train0])
        
        test0 = t[t['place_name_encoded']==x]
        testvacios = pd.concat([testvacios, test0])

In [6]:
#los datos restantes tienen más de 10 entradas en train y al menos alguna en test por barrio
#los junto en el dataframe 'trainaptos'

trainaptos= train[~train['place_name_encoded'].isin(trainvacios['place_name_encoded'])]
testaptos= t[~t['place_name_encoded'].isin(testvacios['place_name_encoded'])]
#trainaptos.info()

In [7]:
#Reviso que quedo bien el split

total = pd.concat([trainaptos, trainvacios])
print('Train:')
print('Registros:      ', len(total), ' = ', len(train))
print('Aptos + Vacios: ', len(trainaptos), '+', len(trainvacios))

print('Place_names:    ', len(train['place_name_encoded'].value_counts()), ' = ', len(total['place_name_encoded'].value_counts()))
print('Vacios + Aptos: ', len(trainvacios['place_name_encoded'].value_counts()), '+', len(trainaptos['place_name_encoded'].value_counts()))
print()


total = pd.concat([testaptos, testvacios])
print('Test:')
print('Registros:      ', len(total), ' = ', len(t))
print('Aptos + Vacios: ', len(testaptos), '+', len(testvacios))

print('Place_names:    ', len(t['place_name_encoded'].value_counts()), ' = ', len(total['place_name_encoded'].value_counts()))
print('Vacios + Aptos: ', len(testvacios['place_name_encoded'].value_counts()), '+', len(testaptos['place_name_encoded'].value_counts()))

Train:
Registros:       609670  =  609670
Aptos + Vacios:  422499 + 187171
Place_names:     899  =  899
Vacios + Aptos:  709 + 190

Test:
Registros:       14166  =  14166
Aptos + Vacios:  14107 + 59
Place_names:     223  =  223
Vacios + Aptos:  33 + 190


In [8]:
trainaptos['place_name_encoded'].value_counts()

615    24108
189    22315
218    14268
834    12929
675    12916
145    11867
16     11257
874    11255
484    10917
381    10896
446    10190
587     8600
42      7539
658     7332
667     7151
671     7068
601     6729
746     6671
241     6593
815     5993
242     5485
46      5094
518     5024
715     4969
694     4778
382     4452
835     4397
728     4177
561     4074
579     4036
       ...  
126      140
644      138
13       129
411      120
581      115
613      113
295       97
709       95
264       94
492       92
425       90
493       86
330       84
420       79
716       75
236       75
768       74
225       71
311       66
799       65
866       63
327       63
14        63
416       61
424       60
421       58
354       56
831       53
650       51
197       50
Name: place_name_encoded, Length: 190, dtype: int64

In [9]:
#armo una funcion que devuelva un dataframe con las predicciones del test y train dados y su id

def RandomForest(test,train):
    train = train.drop('place_name_encoded',1)
    
    trainLX = train.drop('price_aprox_usd', 1)
    trainLY = train[['price_aprox_usd']]
    
    t = test.drop('id',1)
    t = t.drop('place_name_encoded',1)
    
    regr = RandomForestRegressor(min_samples_leaf=1, min_samples_split=2,n_estimators=500, n_jobs = -1,max_features='log2')
    predictions = regr.fit(trainLX, trainLY).predict(t)
    predictions = predictions.flatten()

    ids = test.as_matrix()[:,0].flatten()
    results = pd.DataFrame()
    results['id'] = pd.Series(ids.astype('int32'))
    results['price_usd'] = pd.Series(predictions)
    
    
    return results

#armo el dataframe resultados. Empieza con los test y train 'vacios'
#el resto se hará individualmente por place_name uno por uno posteriormente
resultados = RandomForest(testvacios,trainvacios)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 2 columns):
id           59 non-null int32
price_usd    59 non-null float64
dtypes: float64(1), int32(1)
memory usage: 788.0 bytes


In [10]:
#ahora saco las predicciones para los test y train aptos uno por uno
l = len(properties['place_name_encoded'].value_counts())

for x in range(0,l+2):
    if len(testaptos[testaptos['place_name_encoded']==x])!=0:
        testx = testaptos[testaptos['place_name_encoded']==x]
        trainx = trainaptos[trainaptos['place_name_encoded']==x]
        resx = RandomForest(testx,trainx)
        
        resultados = pd.concat([resultados, resx])

In [11]:
#debe contener la misma cantidad que testing (14166)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14166 entries, 0 to 25
Data columns (total 2 columns):
id           14166 non-null int32
price_usd    14166 non-null float64
dtypes: float64(1), int32(1)
memory usage: 276.7 KB


In [12]:
resultados.set_index(['id'], inplace=True)
resultados.to_csv('predicciones_RF_placename.csv')